In [ ]:
pip install pitch

In [ ]:
#install packages
!pip install librosa
!pip install wave
!pip install scipy
!pip install pydub
!pip install  soundfile
!pip install pygobject
!pip install Audio

In [ ]:
!pip install pyloudnorm

In [ ]:
import math
import wave
import scipy
import librosa
import numpy as np
import librosa.display
import soundfile as sf
from pydub import AudioSegment
from google.colab import files
import matplotlib.pyplot as plt
from IPython.display import Audio
from pydub.utils import db_to_float
from pydub.silence import split_on_silence
from pydub.silence import detect_silence

In [ ]:
import os
import csv
import librosa
import numpy as np
import pitch
import pyloudnorm as pyln

# Function to compute RMSE
def rmse(x):
    return np.sum(x**2)
    pass

folder_path = '/content/temp_data'

# List all files in the folder
files = os.listdir(folder_path)

# Create a CSV file to write the features
csv_file_path = '/content/features.csv'

# Open the CSV file in write mode
with open(csv_file_path, 'w', newline='') as csvfile:
    # Create a CSV writer object
    csv_writer = csv.writer(csvfile)

 # Write the header row with feature names
    header = ['File', 'Pitch', 'RMSE'] + [f'LPC_{i}' for i in range(1, 14)] + [f'MFCC_{i}' for i in range(1, 14)] + ['Intensity','Shimmer', 'Jitter', 'Emotion']
    csv_writer.writerow(header)

    # Iterate over each file
    for file in files:
        # Construct the full path to the file
        file_path = os.path.join(folder_path, file)

        part=file.split('_')
        if part[2] == 'SAD':
            file_emotion = 'sad'
        elif part[2] == 'ANG':
            file_emotion = 'angry'
        elif part[2] == 'DIS':
            file_emotion = 'disgust'
        elif part[2]== 'FEA':
            file_emotion = 'fear'
        elif part[2] == 'HAP':
            file_emotion = 'happy'
        elif part[2] == 'NEU':
            file_emotion = 'neutral'
        else:
            file_emotion = 'Unknown'



        # Load audio file
        x, sr = librosa.load(file_path)

        # Find pitch
        p = pitch.find_pitch(file_path)

        # Compute RMSE
        rmse_value = rmse(x)

        # Compute LPC
        lpc = librosa.lpc(x, order=12)

        # Compute MFCCs
        mfccs = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=13).mean(axis=1)

        data, rate = sf.read(file_path)
        meter = pyln.Meter(rate)
        loudness = meter.integrated_loudness(data)
        positive_intensity = abs(loudness)

        sums=0

        peak=scipy.signal.find_peaks(x,rel_height=0.5)

        #JITTER, SHIMMER, JITTERRAP
        for i in range(1,len(peak[0])-1):
          sums=sums+abs(20*math.log10(peak[0][i+1]/peak[0][i]))

        #SHIMMER
        shimmer=sums/(len(peak[0])-1)
        peakf=abs(np.fft.fft(peak[0]))
        sumps=0
        for i in range(1,len(peakf)-1):
          sumps=sumps+(peakf[i+1]**-1)-(peakf[i]**-1)


        #JITTER
        jitter=sumps/(len(peakf)-1)
        sortedp=np.sort(peak[0])
        sortedf=abs(np.fft.fft(sortedp))
        dif=abs(sortedp[11]-sortedp[15])
        suh=0
        avgabsdiff=(dif)/4
        avgneigh1=(abs(sortedp[6]-sortedp[10]))
        avgneigh2=abs(sortedp[17]-sortedp[22])
        avg=(dif+avgneigh1+avgneigh2)/3

        for i in range(11,16):
          suh=suh+abs(sortedf[i]**-1)
        period=suh/5



        # Write the row to the CSV file
        row = [file, p, rmse_value] + list(lpc) + list(mfccs) + [positive_intensity,shimmer, jitter, file_emotion]
        csv_writer.writerow(row)

# Print a message indicating the completion
print(f'Features have been written to {csv_file_path}')



In [ ]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Initialize lists to store evaluation metrics for each fold
precision_scores = []
recall_scores = []
f1_scores = []
confusion_matrices = []

file_path = '/content/final_features_(1).csv'
data = pd.read_csv(file_path)

# Step 2: Preprocess data if needed

# Step 3: Convert string labels to numeric values
label_encoder = LabelEncoder()
data['Emotion'] = label_encoder.fit_transform(data['Emotion'])

# Step 4: Split into features and labels
X = data.drop(['File','Emotion'], axis=1)
y = data['Emotion']

scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)

# Initialize Random Forest model
rf_model = RandomForestClassifier()

num_folds = 5
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Perform cross-validation
rf_scores = cross_val_score(rf_model, X_standardized, y, cv=kfold, scoring='accuracy')

# Print the accuracy for each fold
for i, score in enumerate(rf_scores, 1):
    print(f'RF Fold {i}: Accuracy = {score:.4f}')

# Print the average accuracy across all folds
print(f'RF Average Accuracy: {rf_scores.mean():.4f}')


SVM

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

def train_model(file_path):
    # Load CSV file
    data = pd.read_csv(file_path)

    # Preprocess data if needed

    # Convert string labels to numeric values
    label_encoder = LabelEncoder()
    data['Emotion'] = label_encoder.fit_transform(data['Emotion'])

    # Split into features and labels
    X = data.drop(['File', 'Emotion'], axis=1)
    y = data['Emotion']

    scaler = StandardScaler()
    X_standardized = scaler.fit_transform(X)

    # Create an SVM model
    svm_model = SVC()

    # Perform K-fold cross-validation
    num_folds = 5
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    # Define variables outside the loop
    y_true_all = []
    y_pred_all = []

    for i, (train_index, test_index) in enumerate(kfold.split(X_standardized)):
        # Split the data into training and testing sets for this fold
        X_train, X_test = X_standardized[train_index], X_standardized[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Train the model on the training set
        svm_model.fit(X_train, y_train)

        # Make predictions on the testing set
        y_pred = svm_model.predict(X_test)

        # Append true and predicted labels for computing overall metrics
        y_true_all.extend(y_test)
        y_pred_all.extend(y_pred)

        # Compute evaluation metrics for this fold
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        confusion_mat = confusion_matrix(y_test, y_pred)


        # Print the evaluation metrics for each fold
        print(f'\nFold {i + 1} Metrics for {file_path}:')
        print(f'Precision = {precision:.4f}')
        print(f'Recall = {recall:.4f}')
        print(f'F1 Score = {f1:.4f}')
        print(f'Confusion Matrix:\n{confusion_mat}')

    # Compute overall metrics using all folds
    overall_precision = precision_score(y_true_all, y_pred_all, average='weighted')
    overall_recall = recall_score(y_true_all, y_pred_all, average='weighted')
    overall_f1 = f1_score(y_true_all, y_pred_all, average='weighted')
    overall_confusion_mat = confusion_matrix(y_true_all, y_pred_all)
    overall_accuracy = accuracy_score(y_true_all, y_pred_all)

    # Print overall metrics
    print(f'\nOverall Metrics for {file_path}:')
    print(f'Overall Precision = {overall_precision:.4f}')
    print(f'Overall Recall = {overall_recall:.4f}')
    print(f'Overall F1 Score = {overall_f1:.4f}')
    print(f'Overall Accuracy = {overall_accuracy:.4f}')
    print(f'Overall Confusion Matrix:\n{overall_confusion_mat}')

# Train the model with the first CSV file
file_path_1 = '/content/features.csv'
train_model(file_path_1)

#Train the model with the second CSV file
file_path_2 = '/content/final_features_(1).csv'
train_model(file_path_2)

#Train the model with the second CSV file
file_path_3 = '/content/final_features_1.csv'
train_model(file_path_3)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Fold 1 Metrics for /content/features.csv:
Precision = 0.3713
Recall = 0.4276
F1 Score = 0.3922
Confusion Matrix:
[[  8   0   0  10   0   0   0   0   1   0   0   0   0   0   0   0   0   0
    0   0   0]
 [  0   3   2   1   0   0   1   0   0   0   0   0   0   0   0   2   7   0
    0   0   0]
 [  0   1   0   7   0   0   1   0   1   0   0   0   0   0   0   2   3   0
    0   0   0]
 [  2   0   0 110   0   0   0   3   1   0   0   9   0   0   0  20   4   0
    0   0   2]
 [  0   0   0   4   0   1   0   1   0   0   0   4   0   0   0   1   3   0
    0   0   2]
 [  0   1   0   0   0   1   0   1   0   0   0   0   0   0   0   0   2   0
    2   0   3]
 [  0   2   0   3   1   1   0   2   0   0   0   1   0   0   0   0   1   0
    0   0   2]
 [  0   0   0  11   0   1   0  46   0   0   0  11   0   0   0  12  22   0
    0   0  31]
 [  1   0   0   2   0   0   0   0   1   0   0   2   0   0   0   2   1   0
    0   0   0]
 [  0   0   0   0   0   1   0   1   0   0   0   3   0   0   0   0   2   0
    0   0  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Fold 2 Metrics for /content/features.csv:
Precision = 0.3806
Recall = 0.4202
F1 Score = 0.3853
Confusion Matrix:
[[  6   0   1   7   0   0   0   0   1   0   0   0   0   0   0   0   0   0
    0   0   0]
 [  0   1   0   1   0   0   0   0   1   0   0   0   0   0   0   1   2   0
    0   0   0]
 [  0   0   0   5   0   0   0   0   0   0   0   2   0   0   0   4   0   0
    0   0   0]
 [  2   0   1 108   0   0   0  15   1   0   0   1   0   0   0  27   2   0
    0   0   1]
 [  1   0   0   0   1   0   1   1   0   0   0   0   0   0   0   3   2   0
    0   0   0]
 [  0   0   0   0   0   1   0   2   0   0   0   0   0   0   0   0   5   0
    0   0   5]
 [  0   1   0   0   0   0   0   2   0   0   0   2   0   0   0   5   2   0
    0   0   1]
 [  0   0   0  16   0   0   0  58   0   0   0   6   0   0   0  10  21   0
    0   0  22]
 [  0   0   0   4   0   0   0   0   2   0   0   3   0   0   0   1   1   0
    0   0   1]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   2   0
    0   0  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Fold 3 Metrics for /content/features.csv:
Precision = 0.3453
Recall = 0.3894
F1 Score = 0.3508
Confusion Matrix:
[[ 4  0  1  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  3  0  0  0  0  0  0  0  1  0  0  0  3  1  0  0  0  0]
 [ 1  1  0  3  0  0  0  0  0  0  0  1  1  0  0  3  1  0  0  0  0]
 [ 1  0  0 97  0  0  0  4  0  0  0  2  0  0  0 21  4  0  0  0  0]
 [ 1  0  0  4  1  0  0  3  0  0  0  2  0  0  0  2  2  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  6  0  1  0  6]
 [ 0  2  0  1  1  0  0  3  0  0  0  3  1  0  0  1  6  0  0  0  0]
 [ 0  0  0 17  0  1  0 54  0  0  0  8  0  0  0 15 40  0  0  0 27]
 [ 1  0  0  5  0  0  0  0  0  0  0  4  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  1  0  0  0  2  0  0  0  0  5  0  0  0  5]
 [ 0  0  0  1  0  0  0  0  0  0  0  3  0  0  0  0  2  0  0  0  8]
 [ 0  1  0 24  0  0  0 10  0  0  0 40  0  0  0 22 27  0  0  0 33]
 [ 0  0  0  5  0  0  0  0  0  0  0  2  1  0  0  3  4  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Fold 4 Metrics for /content/features.csv:
Precision = 0.3460
Recall = 0.4009
F1 Score = 0.3611
Confusion Matrix:
[[ 7  0  1  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  2  0  5  0  0  2  0  0  0  0  3  0  0  0  1  4  0  0  0  0]
 [ 0  1  0  6  0  0  0  1  0  0  0  0  0  0  0  2  0  0  0  0  0]
 [ 0  0  0 90  0  0  0  7  0  0  0 10  0  0  0 36  3  0  0  0  1]
 [ 0  0  0  1  0  0  0  2  0  0  0  1  0  0  0  3  1  0  0  0  3]
 [ 0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  6  0  0  0  4]
 [ 0  1  0  3  1  0  0  0  0  0  0  0  0  0  0  3  3  0  0  0  1]
 [ 0  0  0 10  0  0  0 55  0  0  0 12  0  0  0 10 35  0  0  0 28]
 [ 3  0  1  6  0  0  0  0  0  0  0  5  0  0  0  4  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  7  0  0  0  8]
 [ 0  0  1  1  0  0  0  1  0  0  0  0  0  0  0  1  5  0  0  0  3]
 [ 0  0  0 16  0  0  0 13  0  0  0 39  0  0  0 29 23  0  0  0 38]
 [ 0  0  0  6  0  0  0  0  1  0  0  1  0  0  0  2  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Fold 5 Metrics for /content/features.csv:
Precision = 0.3267
Recall = 0.3804
F1 Score = 0.3356
Confusion Matrix:
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1]
 [ 0  2  0  0  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  3  0  0  3  1  0  0  0  0  0  0  0  5  7  0  0  0  0]
 [ 0  0  0  1 11  0  0  1  0  0  0  0  1  0  0  0  2  2  0  0  0  1]
 [ 0  1  0  0 90  0  0  0  7  0  0  0  7  0  0  0 22  5  0  0  0  0]
 [ 0  0  0  0  2  0  0  2  3  0  0  0  2  1  0  0  2  0  0  0  0  2]
 [ 0  0  0  0  0  0  1  1  2  0  0  0  3  0  0  0  0  2  0  0  0  9]
 [ 0  0  0  0  1  0  0  0  2  0  0  0  0  0  0  0  1  4  0  0  0  2]
 [ 0  0  0  0 16  0  0  0 56  0  0  0  3  0  0  0 15 24  0  0  0 24]
 [ 0  1  0  0  4  0  0  0  0  2  0  0  5  0  0  0  0  0  0  0  0  2]
 [ 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  5  0  0  0 10]
 [ 0  0  0  1  1  0  0  0  2  0  0  0  0  0  0  0  1  4  0  0  0  2]
 [ 0  1  0  0 15  0  0  0 17  0  0  0 27  0  0  0 32 17  0